<a href="https://colab.research.google.com/github/Not-Pace/Genre-Classifier/blob/main/Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The Non-Image Approach**

Before this, we tried 3 different CNNs, one using Transfer learning and images from the dataset, the other without transfer learning and a third taking a more direct approach, converting the audios into mel-spectrograms and analysing them as images. (The second and third model are essentially the same thing, but the data is processed differently, and that's what we tried to gauge with that.)

In this approach, we're going to use the features that were extracted from the dataset songs like Tempo, Zero Crossing, MFCC mean and variance, etc.

In [2]:
import pandas as pd
import numpy as np
import tarfile
from six.moves import urllib
import matplotlib.pyplot as plt
%matplotlib inline

import os
from PIL import Image
import pathlib
import csv

import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import regularizers

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

#Confusion Matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import keras

In [3]:
from google.colab import drive

In [4]:
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [5]:
dataset_path = '/content/gdrive/My Drive/Genre-Classifier/Data/features_30_sec.csv'

In [6]:
seed = 7
np.random.seed(seed)

master_data = pd.read_csv(dataset_path) 
# master_data = master_data.drop(master_data.columns[range(21,59)], axis = 1)
master_data = master_data.drop("filename", axis = 1)

labels = ['blues',
          'classical',
          'country',
          'disco',
          'hiphop',
          'jazz',
          'metal',
          'pop',
          'reggae',
          'rock']

master_data.head()

,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,zero_crossing_rate_mean,zero_crossing_rate_var,harmony_mean,harmony_var,perceptr_mean,perceptr_var,tempo,mfcc1_mean,mfcc1_var,mfcc2_mean,mfcc2_var,mfcc3_mean,mfcc3_var,mfcc4_mean,mfcc4_var,mfcc5_mean,mfcc5_var,mfcc6_mean,mfcc6_var,mfcc7_mean,mfcc7_var,mfcc8_mean,mfcc8_var,mfcc9_mean,mfcc9_var,mfcc10_mean,mfcc10_var,mfcc11_mean,mfcc11_var,mfcc12_mean,mfcc12_var,mfcc13_mean,mfcc13_var,mfcc14_mean,mfcc14_var,mfcc15_mean,mfcc15_var,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,661794,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,3805.839606,9.015054e+05,0.083045,0.000767,-4.529724e-05,0.008172,0.000008,0.005698,123.046875,-113.570648,2564.207520,121.571793,295.913818,-19.168142,235.574432,42.366421,151.106873,-6.364664,167.934799,18.623499,89.180840,-13.704891,67.660492,15.343150,68.932579,-12.274110,82.204201,10.976572,63.386311,-8.326573,61.773094,8.803792,51.244125,-3.672300,41.217415,5.747995,40.554478,-5.162882,49.775421,0.752740,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035,blues
1,661794,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,3550.522098,2.977893e+06,0.056040,0.001448,1.395807e-04,0.005099,-0.000178,0.003063,67.999589,-207.501694,7764.555176,123.991264,560.259949,8.955127,572.810913,35.877647,264.506104,2.907320,279.932922,21.510466,156.477097,-8.560436,200.849182,23.370686,142.555954,-10.099661,166.108521,11.900497,104.358612,-5.555639,105.173630,5.376327,96.197212,-2.231760,64.914291,4.220140,73.152534,-6.012148,52.422142,0.927998,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282,blues
2,661794,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,3042.260232,7.840345e+05,0.076291,0.001007,2.105576e-06,0.016342,-0.000019,0.007458,161.499023,-90.722595,3319.044922,140.446304,508.765045,-29.093889,411.781219,31.684334,144.090317,-13.984504,155.493759,25.764742,74.548401,-13.664875,106.981827,11.639934,106.574875,-11.783643,65.447945,9.718760,67.908859,-13.133803,57.781425,5.791199,64.480209,-8.907628,60.385151,-1.077000,57.711136,-9.229274,36.580986,2.451690,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025,blues
3,661794,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,2184.745799,1.493194e+06,0.033309,0.000423,4.583644e-07,0.019054,-0.000014,0.002712,63.024009,-199.544205,5507.517090,150.090897,456.505402,5.662678,257.161163,26.859079,158.267303,1.771399,268.034393,14.234031,126.794128,-4.832006,155.912079,9.286494,81.273743,-0.759186,92.114090,8.137607,71.314079,-3.200653,110.236687,6.079319,48.251999,-2.480174,56.799400,-1.079305,62.289902,-2.870789,51.651592,0.780874,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339,blues
4,661794,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,3579.757627,1.572978e+06,0.101461,0.001954,-1.756129e-05,0.004814,-0.000010,0.003094,135.999178,-160.337708,5195.291992,126.219635,853.784729,-35.587811,333.792938,22.148071,193.456100,-32.478600,336.276825,10.852294,134.831573,-23.352329,93.257095,0.498434,124.672127,-11.793437,130.073349,1.207256,99.675575,-13.088418,80.254066,-2.813867,86.430626,-6.933385,89.555443,-7.552725,70.943336,-9.164666,75.793404,-4.520576,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160,blues


In [7]:
master_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 59 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   length                   1000 non-null   int64  
 1   chroma_stft_mean         1000 non-null   float64
 2   chroma_stft_var          1000 non-null   float64
 3   rms_mean                 1000 non-null   float64
 4   rms_var                  1000 non-null   float64
 5   spectral_centroid_mean   1000 non-null   float64
 6   spectral_centroid_var    1000 non-null   float64
 7   spectral_bandwidth_mean  1000 non-null   float64
 8   spectral_bandwidth_var   1000 non-null   float64
 9   rolloff_mean             1000 non-null   float64
 10  rolloff_var              1000 non-null   float64
 11  zero_crossing_rate_mean  1000 non-null   float64
 12  zero_crossing_rate_var   1000 non-null   float64
 13  harmony_mean             1000 non-null   float64
 14  harmony_var              

In [8]:
master_dataset = master_data.values
X = master_dataset[:,0:58].astype(float)
Y = master_dataset[:,58]
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
dummy = np_utils.to_categorical(encoded_Y)
dummy[0]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [9]:
sc = StandardScaler()
X = sc.fit_transform(X)

In [10]:
def baseline_model(optimizer='adam', init='glorot_uniform'):
  # create model
  model = Sequential()
  model.add(Dense(58, kernel_initializer=init, activation='relu'))
  model.add(Dense(29, kernel_initializer=init, activation='relu'))
  # model.add(Dense(20, kernel_initializer=init, activation='relu'))
  model.add(Dense(10, kernel_initializer=init, activation='softmax'))
  # Compile model
  model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  return model

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, dummy, test_size=0.33, random_state=7)
model = KerasClassifier(build_fn=baseline_model, verbose=0)
# grid search epochs, batch size and optimizer
optimizers = ['rmsprop', 'adam']
init = ['glorot_uniform', 'normal', 'uniform']
epochs = [15, 30, 45]
batches = [5, 10, 20]
param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=init)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.725373 using {'batch_size': 5, 'epochs': 30, 'init': 'glorot_uniform', 'optimizer': 'adam'}
0.708955 (0.015654) with: {'batch_size': 5, 'epochs': 15, 'init': 'glorot_uniform', 'optimizer': 'rmsprop'}
0.695522 (0.023314) with: {'batch_size': 5, 'epochs': 15, 'init': 'glorot_uniform', 'optimizer': 'adam'}
0.662687 (0.034812) with: {'batch_size': 5, 'epochs': 15, 'init': 'normal', 'optimizer': 'rmsprop'}
0.692537 (0.027277) with: {'batch_size': 5, 'epochs': 15, 'init': 'normal', 'optimizer': 'adam'}
0.619403 (0.040049) with: {'batch_size': 5, 'epochs': 15, 'init': 'uniform', 'optimizer': 'rmsprop'}
0.659701 (0.056164) with: {'batch_size': 5, 'epochs': 15, 'init': 'uniform', 'optimizer': 'adam'}
0.698507 (0.018034) with: {'batch_size': 5, 'epochs': 30, 'init': 'glorot_uniform', 'optimizer': 'rmsprop'}
0.725373 (0.035130) with: {'batch_size': 5, 'epochs': 30, 'init': 'glorot_uniform', 'optimizer': 'adam'}
0.697015 (0.021422) with: {'batch_size': 5, 'epochs': 30, 'init': 'normal', 'o

If you didn't read through the entire analysis of that, that's alright. The GridSearch cross validation recommends 45 epochs, a batch size of 20, glorot_uniform initialisation and the adam optimiser for the best results. Setting those values in on our model, our baseline accuracy jumps up from the mid 60s to a solid 73.2%. While this value may not be particularly high either, it is an improvement over all the CNNs that we tried implementing to solve the problem.

In [12]:
model = KerasClassifier(build_fn=baseline_model, epochs=30, batch_size=5, verbose=0)
kfold = KFold(n_splits=8, shuffle=True,  random_state=seed)
results = cross_val_score(model, X, dummy, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 73.20% (2.71%)
